In [ ]:
from mp_api.client import MPRester
import pandas as pd

API_KEY = "4IeHY5jVcrgiKXNuAo6Jgs7yC0Z3hsli"

"""try:
    with MPRester(API_KEY) as mpr:
        # Search for materials with a band gap greater than 0.5 eV and check stability
        docs = mpr.materials.summary.search(
            band_gap=[0.5, None], # band_gap greater than or equal to 0.5 eV
            is_stable=True,       # material is stable (on the convex hull)
            fields=["material_id", "formula_pretty", "band_gap", "is_stable"] # request specific fields
        )

        print(f"Found {len(docs)} stable materials with a band gap >= 0.5 eV.\n")

        # Iterate through the results and print the data
        print("{:<15} {:<15} {:<15} {:<10}".format("Material ID", "Formula", "Band Gap (eV)", "Is Stable"))
        print("-" * 55)
        for doc in docs[:10]: # Print the first 10 results
            print("{:<15} {:<15} {:<15.3f} {:<10}".format(
                doc.material_id,
                doc.formula_pretty,
                doc.band_gap,
                doc.is_stable
            ))

except Exception as e:
    print(f"An error occurred: {e}")"""

In [ ]:
from mp_api.client import MPRester
API_KEY = "4IeHY5jVcrgiKXNuAo6Jgs7yC0Z3hsli"
try:


    with MPRester(API_KEY) as mpr:
        # Query for Li-based cathode materials with high energy density
        docs = mpr.materials.insertion_electrodes.search(
            working_ion="Li",
            fields=[
                "battery_id", "formula_discharge", "average_voltage"
                , "energy_grav", "energy_vol", "capacity_grav", "capacity_vol", "stability_charge", "fracA_charge", "stability_charge", "max_delta_volume", "stability_discharge", "fracA_discharge"
            ]
        )
        """
        battey_id = id for specific batteyr material system. (base material id + working ion being inserted)

        formulaDischarge = chem formula in fully discharged state

        avgVoltage = average voltage this material would produce during discharge cycle
                note: negative voltage would be good for anode, pos value good for cathode

        capacityGrav = mAh/g (milliamp-hours per gram), (amount of electrical charge stored per unit of mass.)

        energy_grav = energy amount stored per unit of weight
            energy density = Voltage x Capacity

        energy_vol (Volumetric Energy Density) = measures how much energy is stored per unit of space (volume)

        stability_charge((Electrochemical Stability) = measures the thermodynamic stability of the material when in fully charged state (with the Lithium removed).
            Research Insight: If this value were high (e.g., > 0.100) - relatively unstable for battery

        """

        # Convert to pandas for research analysis
        import pandas as pd
        df = pd.DataFrame([doc.dict() for doc in docs])
        print(df)

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
stable = df[df["stability_charge"] < 0.100]
stable.head()

In [ ]:
import matplotlib.pyplot as plt


df['average_voltage'].hist(bins=20)
plt.xlabel("Average voltage")
plt.ylabel("Number of electric charges")
plt.title("Practice 1")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='capacity_grav', y='average_voltage',
                hue='energy_grav', size='energy_grav', palette='viridis')
plt.title("Voltage vs. Capacity (Size = Energy Density)")
plt.xlabel("Capacity (mAh/g)")
plt.ylabel("Average Voltage (V)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

"""
Most instances fall within 0 - 200 capacity range and 0 - 5 Average Voltage range"""

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='capacity_grav', y='stability_charge',
                hue='energy_grav', size='energy_grav', palette='viridis')
plt.title("Stability vs. capacity (Size = Energy Density)")
plt.xlabel("capacity")
plt.ylabel("Stability Charge")
#plt.xlim(300, 1400)
plt.ylim(0, 0.13)
plt.grid(True, linestyle='--', alpha=0.6)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='capacity_grav', y='max_delta_volume',
                hue='energy_grav', size='energy_grav', palette='viridis')
plt.title("Stability vs. Capacity (Size = Energy Density)")
plt.xlabel("Capacity (mAh/g)")
plt.ylabel("Energy Density")
plt.xlim(100, 1400)
plt.ylim(0, 0.3)
plt.grid(True, linestyle='--', alpha=0.6)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Select only the numeric columns relevant to battery performance
cols_to_analyze = ['average_voltage', 'capacity_grav', 'energy_grav',
                   'stability_discharge', 'max_delta_volume', 'fracA_discharge']

# Calculate the Pearson correlation
correlation_matrix = df[cols_to_analyze].corr()

correlation_matrix.head()

In [ ]:
# Example: Only analyze 'realistic' stable materials
df_clean = df[(df['stability_charge'] < 0.2) & (df['average_voltage'] > 0)]
correlation_clean = df_clean[cols_to_analyze].corr()


In [ ]:
#- Compare correlation of energy density with capacity

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_clean, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Battery Properties")
plt.show()

In [ ]:
"""
INSIGHT:
Negative correlation between average voltage with stability_discharge, fracA_discharge, max_delta_volume"""

In [ ]:
cols_to_analyze1 = ['average_voltage', 'stability_charge',
                   'stability_discharge', 'max_delta_volume', 'fracA_charge', 'fracA_discharge']

# Calculate the Pearson correlation
correlation_clean = df_clean[cols_to_analyze1].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_clean, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Battery Properties for Lifespan")
plt.show()

In [ ]:
df.describe()

In [ ]:
#Use to display info on specific battery material id

with MPRester(API_KEY) as mpr:
    # Query for the specific battery_id
    # Note: Use mpr.materials.insertion_electrodes to avoid deprecation warnings
    docs = mpr.materials.insertion_electrodes.search(battery_ids="mp-568806_Li")

    if docs:
        # Convert the first (and only) matching document to a dictionary
        battery_data = docs[0].dict()

        # Method 1: Display as a readable DataFrame (Transposed for easier reading)
        df7 = pd.DataFrame([battery_data]).T
        df7.columns = ["Value"]
        print(df7)

        # Method 2: Print as a formatted JSON-like dictionary
        # import json
        # print(json.dumps(battery_data, indent=4, default=str))
    else:
        print("No battery data found for this ID.")

In [ ]:
for field in mpr.materials.insertion_electrodes.available_fields:
    print(field)
# Displays all available fields